# PySR match analysis

Load PySR results across budgets, check symbolic exact matches with caching, and inspect high-R2 non-matches.


In [1]:
import json
import hashlib
from pathlib import Path

import pandas as pd

import evaluation
from utils import load_srbench_dataset

RESULTS_ROOT = Path('.')
RESULTS_DIRS = [
    RESULTS_ROOT / 'results_pysr_1e3',
    RESULTS_ROOT / 'results_pysr_1e4',
    RESULTS_ROOT / 'results_pysr_1e5',
    RESULTS_ROOT / 'results_pysr_1e6',
    RESULTS_ROOT / 'results_pysr_1e7',
    RESULTS_ROOT / 'results_pysr_1e8',
]

CACHE_PATH = RESULTS_ROOT / 'caches/pysr_symbolic_match_cache.json'


ModuleNotFoundError: No module named 'evaluation'

In [2]:
def load_symbolic_cache(path=CACHE_PATH):
    if path.exists():
        try:
            return json.loads(path.read_text())
        except Exception:
            return {}
    return {}


def save_symbolic_cache(cache, path=CACHE_PATH):
    path.write_text(json.dumps(cache, indent=2, sort_keys=True))


def _cache_key(dataset, equation, ground_truth, var_names):
    payload = {
        'dataset': dataset,
        'equation': equation,
        'ground_truth': ground_truth,
        'var_names': var_names,
    }
    raw = json.dumps(payload, sort_keys=True)
    return hashlib.sha256(raw.encode('utf-8')).hexdigest()


def check_symbolic_match_cached(dataset, equation, ground_truth, var_names, cache):
    key = _cache_key(dataset, equation, ground_truth, var_names)
    if key in cache:
        return cache[key]
    result = evaluation.check_pysr_symbolic_match(equation, ground_truth, var_names)
    cache[key] = result
    return result


def load_results_dir(results_dir):
    rows = []
    budget = results_dir.name.replace('results_pysr_', '')
    for path in sorted(results_dir.glob('*_results.json')):
        data = json.loads(path.read_text())
        rows.append({
            'budget': budget,
            'dataset': data.get('dataset'),
            'test_r2': data.get('test_r2'),
            'test_mse': data.get('test_mse'),
            'n_features': data.get('n_features'),
            'best_equation': data.get('best_equation'),
            'result_path': str(path),
        })
    return rows


def load_all_results(results_dirs=RESULTS_DIRS):
    rows = []
    for results_dir in results_dirs:
        if results_dir.exists():
            rows.extend(load_results_dir(results_dir))
    return pd.DataFrame(rows)


def get_ground_truth_and_vars(dataset):
    _, _, ground_truth = load_srbench_dataset(dataset, max_samples=10)
    var_names = evaluation.get_dataset_var_names(dataset)
    return ground_truth, var_names


def ground_truth_complexity(ground_truth, var_names):
    expr = evaluation.parse_ground_truth_formula(ground_truth, var_names)
    return evaluation.complexity(expr)


In [3]:
symbolic_cache = load_symbolic_cache()

results_df = load_all_results()
results_df.head()


,budget,dataset,test_r2,test_mse,n_features,best_equation,result_path
0,1e3,feynman_III_10_19,0.775130,12.031329,4,1.21507081493463*By + 1.21507081493463*exp(sqr...,results_pysr_1e3/feynman_III_10_19_results.json
1,1e3,feynman_III_12_43,0.769507,0.163238,2,(0.0380710885570596*h**2)*n,results_pysr_1e3/feynman_III_12_43_results.json
2,1e3,feynman_III_13_18,0.610443,130184.334901,4,-E_n + d**2 + d + k**2 + k + k + (d + d - 3.09...,results_pysr_1e3/feynman_III_13_18_results.json
3,1e3,feynman_III_14_14,0.412521,42.975845,5,(I_0/T + exp(sqrt(I_0)))*(-kb + q + 0.856112),results_pysr_1e3/feynman_III_14_14_results.json
4,1e3,feynman_III_15_12,0.240339,22.235178,3,sqrt(U + exp(U)),results_pysr_1e3/feynman_III_15_12_results.json


In [4]:
# Enrich with ground truth, variable names, and symbolic matches.

def _get_progress(iterable, total=None):
    try:
        from tqdm.auto import tqdm
        return tqdm(iterable, total=total)
    except Exception:
        return iterable


def enrich_results(df, cache):
    ground_truth_by_dataset = {}
    var_names_by_dataset = {}

    matches = []
    ground_truths = []
    var_names_list = []

    iterable = _get_progress(df.iterrows(), total=len(df))
    for _, row in iterable:
        dataset = row['dataset']
        if dataset not in ground_truth_by_dataset:
            ground_truth, var_names = get_ground_truth_and_vars(dataset)
            ground_truth_by_dataset[dataset] = ground_truth
            var_names_by_dataset[dataset] = var_names

        ground_truth = ground_truth_by_dataset[dataset]
        var_names = var_names_by_dataset[dataset]

        match_result = check_symbolic_match_cached(
            dataset,
            row['best_equation'],
            ground_truth,
            var_names,
            cache,
        )

        matches.append(bool(match_result.get('match')))
        ground_truths.append(ground_truth)
        var_names_list.append(var_names)

    df = df.copy()
    df['ground_truth'] = ground_truths
    df['var_names'] = var_names_list
    df['symbolic_match'] = matches
    return df

results_df = enrich_results(results_df, symbolic_cache)
save_symbolic_cache(symbolic_cache)

results_df.head()


  0%|          | 0/742 [00:00<?, ?it/s]

,budget,dataset,test_r2,test_mse,n_features,best_equation,result_path,ground_truth,var_names,symbolic_match
0,1e3,feynman_III_10_19,0.775130,12.031329,4,1.21507081493463*By + 1.21507081493463*exp(sqr...,results_pysr_1e3/feynman_III_10_19_results.json,E_n = mom*sqrt(Bx**2+By**2+Bz**2),"[mom, Bx, By, Bz]",False
1,1e3,feynman_III_12_43,0.769507,0.163238,2,(0.0380710885570596*h**2)*n,results_pysr_1e3/feynman_III_12_43_results.json,L = n*(h/(2*pi)),"[n, h]",False
2,1e3,feynman_III_13_18,0.610443,130184.334901,4,-E_n + d**2 + d + k**2 + k + k + (d + d - 3.09...,results_pysr_1e3/feynman_III_13_18_results.json,v = 2*E_n*d**2*k/(h/(2*pi)),"[E_n, d, k, h]",False
3,1e3,feynman_III_14_14,0.412521,42.975845,5,(I_0/T + exp(sqrt(I_0)))*(-kb + q + 0.856112),results_pysr_1e3/feynman_III_14_14_results.json,I = I_0*(exp(q*Volt/(kb*T))-1),"[I_0, q, Volt, kb, T]",False
4,1e3,feynman_III_15_12,0.240339,22.235178,3,sqrt(U + exp(U)),results_pysr_1e3/feynman_III_15_12_results.json,E_n = 2*U*(1-cos(k*d)),"[U, k, d]",False


In [5]:
# Helper to inspect 1e8 high-R2 non-matches.

def show_high_r2_nonmatches(df, budget='1e8'):
    subset = df[(df['budget'] == budget) & (0.999 < df['test_r2']) & (~df['symbolic_match'])].copy()
    if subset.empty:
        print('No rows match the filter.')
        return subset

    complexities = []
    for _, row in subset.iterrows():
        complexities.append(ground_truth_complexity(row['ground_truth'], row['var_names']))

    subset['ground_truth_complexity'] = complexities
    subset = subset.sort_values('ground_truth_complexity', ascending=True)

    for _, row in subset.iterrows():
        print(f"{row['dataset']} | complexity={row['ground_truth_complexity']} | R2={row['test_r2']:.6f}")
        print(f"  GT:  {row['ground_truth']}")
        print(f"  PySR:{row['best_equation']}")
        print()

    return subset

high_r2_nonmatches = show_high_r2_nonmatches(results_df)


feynman_III_8_54 | complexity=11 | R2=1.000000
  GT:  prob = sin(E_n*t/(h/(2*pi)))**2
  PySR:sin((-E_n*t + h)/((h/6.2831855)))**2

feynman_II_11_27 | complexity=13 | R2=1.000000
  GT:  Pol = n*alpha/(1-(n*alpha/3))*epsilon*Ef
  PySR:Ef*1.7962372*alpha*epsilon*n/(n*alpha*(-0.59874576) - 1*(-1.7962372))

feynman_II_11_28 | complexity=13 | R2=1.000000
  GT:  theta = 1+n*alpha/(1-(n*alpha/3))
  PySR:-0.00929176*alpha**2*n**2*sin(alpha*n*(alpha + n + 0.98740596)) + sqrt(exp(1.7783664*alpha*n) - 0.2099078) + 0.111132495

feynman_II_35_21 | complexity=13 | R2=0.999999
  GT:  M = n_rho*mom*tanh(mom*B/(kb*T))
  PySR:0.517114920595421*n_rho*(mom - 0.037284777*T*kb/B)*(cos(0.11707609 - 2.6165955*exp(-0.893943044206824*B*mom/(T*kb))) + 0.9508448)

feynman_I_13_4 | complexity=13 | R2=0.999999
  GT:  K = 1/2*m*(v**2+u**2+w**2)
  PySR:m*(0.308051141054603*v**2 + (-sin(cos(u))**2 + cos(cos(w))**2)*(-0.33330092) + (u + w - 2.1832488)*1.9351565 + cos(u) + cos(w))*1.623136

feynman_test_5 | complexity=13

In [6]:
# for 1e3 up to 1e8, calculate percent of tasks that have exact match, and percent of tasks that have R^2 = 1.0
summary_rows = []
for budget in ['1e3', '1e4', '1e5', '1e6', '1e7', '1e8']:
    subset = results_df[results_df['budget'] == budget]
    total = len(subset)
    if total == 0:
        continue
    exact_matches = subset['symbolic_match'].sum()
    perfect_r2 = (subset['test_r2'] == 1.0).sum()
    summary_rows.append({
        'budget': budget,
        'total_tasks': total,
        'exact_match_count': exact_matches,
        'exact_match_percent': exact_matches / total * 100,
        'perfect_r2_count': perfect_r2,
        'perfect_r2_percent': perfect_r2 / total * 100,
    })

In [ ]:
# print results as table
summary_df = pd.DataFrame(summary_rows)
summary_df

,budget,total_tasks,exact_match_count,exact_match_percent,perfect_r2_count,perfect_r2_percent
0,1e3,124,2,1.612903,1,0.806452
1,1e4,124,4,3.225806,3,2.419355
2,1e5,124,31,25.000000,18,14.516129
3,1e6,124,59,47.580645,36,29.032258
4,1e7,124,70,56.451613,43,34.677419
5,1e8,122,74,60.655738,52,42.622951


In [ ]:
# sort tasks by ground truth complexity. print tasks for 1e8 that got high R^2 but no symbolic match
high_r2_nonmatches = show_high_r2_nonmatches(results_df, budget='1e8')

In [1]:
from utils import load_json
combined = load_json('results/pysr_1e6_2/slurm_pysr/eval_0000/combined.json')
r2_s = [res['r2_score'] for res in combined]
avg_r2 = sum(r2_s) / len(r2_s)
print(avg_r2)

0.964323061674723
